In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds

# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [2]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [3]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sta314 (takim). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'augmentation': {
          'values': ['none', 'light', 'heavy']
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.001},
    'batch_size': {
        'value': 64},
    'kernel_size': {
          'value': '5x5'},
    'net_filter_size': {
          'value': 32},
    'net_n': {
          'value': 3},
    'reg_alpha': {
          'value': 0.0001},
    'normalization': {
          'value': False}
    })

In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'augmentation': {'values': ['none', 'light', 'heavy']},
                'batch_size': {'value': 64},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'kernel_size': {'value': '5x5'},
                'learning_rate': {'value': 0.001},
                'net_filter_size': {'value': 32},
                'net_n': {'value': 3},
                'normalization': {'value': False},
                'reg_alpha': {'value': 0.0001}}}


In [6]:
# sweep_id = wandb.sweep(sweep_config, project="zayif-test")
sweep_id = wandb.sweep(sweep_config, project="CIFAR-10_Classification")

Create sweep with ID: zt6k7al0
Sweep URL: https://wandb.ai/takim/CIFAR-10_Classification/sweeps/zt6k7al0


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Lambda, Add, Input, GlobalAveragePooling2D, Flatten, Dense, Softmax
from tensorflow.keras import Model
from tensorflow.keras.regularizers import l2

def ResidualBlock(x, filter_size, is_switch_block, kernel_size, reg_alpha):

    # note that if is_switch_block true, it means that output will not be the same as the input
    # so while merging the residual connection, we need to adapt to it
    # this adaptation could be with a conv layer, or a simple downsampling + padding is enough.

    x_skip = x # save original input to the block

    if not is_switch_block:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    else:
        x = Conv2D(filter_size, kernel_size=kernel_size, strides=(2, 2), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = BatchNormalization()(x)

    if is_switch_block: # takes every second element to half(v) spatial dimension and then adds padding to each side for matching filter (last) dimension
        x_skip = Lambda(lambda x: tf.pad(x[:, ::2, ::2, :], tf.constant([[0, 0,], [0, 0], [0, 0], [filter_size//4, filter_size//4]]), mode="CONSTANT"))(x_skip)

    x = Add()([x, x_skip])
    x = ReLU()(x)

    return x

def ResidualBlocks(x, filter_size, n, kernel_size, reg_alpha):
    for group in range(3): # a stack of 6n layers, 3×3 convolutions, feature maps of sizes {4fs, 2fs, fs}, 2n layers for each size
        for block in range(n):
            if group > 0 and block == 0: # double filter size
                filter_size *= 2
                is_switch_block = True
            else:
                is_switch_block = False
                
            x = ResidualBlock(x, filter_size, is_switch_block, kernel_size, reg_alpha)

    return x

def create_model(config):

    filter_size = config['net_filter_size']
    n = config['net_n']
    kernel_size = (3, 3) if config['kernel_size'] == '3x3' else (5, 5)

    reg_alpha = config['reg_alpha']

    inputs = Input(shape=(32, 32, 3))
    x = Conv2D(filter_size, kernel_size=kernel_size, strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = ResidualBlocks(x, filter_size, n, kernel_size, reg_alpha)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(10)(x)
    outputs = Softmax()(x)

    model = Model(inputs, outputs, name=f"ResNet-{n*6+2}")
    return model

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        do_normalization = config['normalization']
        do_augmentation = config['augmentation'] != 'none'

        x_train_to_use = (x_train.astype('float32') / 255) if do_normalization else x_train
        x_test_to_use = (x_test.astype('float32') / 255) if do_normalization else x_test

        tf.keras.backend.clear_session()
        model = create_model(config)
        model.compile(
            optimizer = Adam(learning_rate=config["learning_rate"]),
            loss = "categorical_crossentropy",
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        if not do_augmentation:
            history = model.fit(x_train_to_use, y_train,
                                epochs=config["epochs"],
                                batch_size=config["batch_size"],
                                validation_split=0.1,
                                callbacks=[
                                    WandbMetricsLogger(log_freq='epoch'),
                                    early_stopping
                                ], verbose=1
                                )
        else:
            if config['augmentation'] == 'light':
                datagen = ImageDataGenerator(
                    rotation_range=20,
                    horizontal_flip=True,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    fill_mode='nearest'
                )
            else:
                datagen = ImageDataGenerator(
                    rotation_range=40,
                    horizontal_flip=True,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.1,
                    zoom_range=0.1,
                    fill_mode='nearest'
                )

            x_tr, x_vl, y_tr, y_vl = train_test_split(x_train_to_use, y_train, test_size=0.1, random_state=42)

            train_datagen = datagen.flow(x_tr, y_tr, batch_size=config["batch_size"])
            history = model.fit(train_datagen,
                                epochs=config["epochs"],
                                batch_size=config["batch_size"],
                                validation_data=(x_vl, y_vl),
                                callbacks=[
                                    WandbMetricsLogger(log_freq='epoch'),
                                    early_stopping
                                ], verbose=1
                                )
            
        
        test_stats = model.evaluate(x_test_to_use, y_test)
        wandb.log({"test_loss": test_stats[0]})
        wandb.log({"test_acc": test_stats[1]})

        val_loss_history = history.history['val_loss']
        val_acc_history = history.history['val_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [9]:
# wandb.agent(sweep_id, train, count=1)
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: p6hhejtu with config:
wandb: 	augmentation: none
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: 5x5
wandb: 	learning_rate: 0.001
wandb: 	net_filter_size: 32
wandb: 	net_n: 3
wandb: 	normalization: False
wandb: 	reg_alpha: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 39s 51ms/step - loss: 1.5230 - accuracy: 0.5016 - top@3_accuracy: 0.8133 - val_loss: 1.3429 - val_accuracy: 0.5672 - val_top@3_accuracy: 0.8686
Epoch 2/100
704/704 [==============================] - 39s 55ms/step - loss: 1.0519 - accuracy: 0.6805 - top@3_accuracy: 0.9132 - val_loss: 1.7686 - val_accuracy: 0.5012 - val_top@3_accuracy: 0.8206
Epoch 3/100
704/704 [==============================] - 42s 60ms/step - loss: 0.8733 - accuracy: 0.7512 - top@3_accuracy: 0.9403 - val_loss: 1.4278 - val_accuracy: 0.6212 - val_top@3_accuracy: 0.8510
Epoch 4/100
704/704 [==============================] - 46s 65ms/step - loss: 0.7908 - accuracy: 0.7846 - top@3_accuracy: 0.9553 - val_loss: 1.3443 - val_accuracy: 0.6006 - val_top@3_accuracy: 0.8884
Epoch 5/100
704/704 [==============================] - 50s 71ms/step - loss: 0.7324 - accuracy: 0.8139 - top@3_accuracy: 0.9644 - val_loss: 0.9462 - val_accuracy: 0.7532 - val_top@3_accuracy: 0.9260
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇█████████████
epoch/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▅▆▆▇▇▇▇▇▇██████████████████
epoch/val_accuracy,▂▁▄▃▆▇▆▄▇▆▅▆▇▅▆▇▆█▆▇▇█▇▆▇▇█▅
epoch/val_loss,▅█▅▅▂▂▃▇▂▂▄▃▃▇▃▃▄▁▄▃▄▂▄▅▄▄▂█
epoch/val_top@3_accuracy,▃▁▂▄▆▇▇▃▇▇▆▆▇▄▆█▆█▆▇▇█▇▆▇▇█▆
test_acc,▁


wandb: Agent Starting Run: n11y8zac with config:
wandb: 	augmentation: light
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: 5x5
wandb: 	learning_rate: 0.001
wandb: 	net_filter_size: 32
wandb: 	net_n: 3
wandb: 	normalization: False
wandb: 	reg_alpha: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 50s 69ms/step - loss: 1.6803 - accuracy: 0.4422 - top@3_accuracy: 0.7754 - val_loss: 1.7068 - val_accuracy: 0.4326 - val_top@3_accuracy: 0.7656
Epoch 2/100
704/704 [==============================] - 55s 79ms/step - loss: 1.2761 - accuracy: 0.5936 - top@3_accuracy: 0.8727 - val_loss: 1.5201 - val_accuracy: 0.5528 - val_top@3_accuracy: 0.8584
Epoch 3/100
704/704 [==============================] - 56s 80ms/step - loss: 1.0889 - accuracy: 0.6696 - top@3_accuracy: 0.9075 - val_loss: 1.3501 - val_accuracy: 0.5988 - val_top@3_accuracy: 0.8982
Epoch 4/100
704/704 [==============================] - 56s 80ms/step - loss: 0.9929 - accuracy: 0.7116 - top@3_accuracy: 0.9258 - val_loss: 0.9923 - val_accuracy: 0.7160 - val_top@3_accuracy: 0.9150
Epoch 5/100
704/704 [==============================] - 56s 80ms/step - loss: 0.9282 - accuracy: 0.7366 - top@3_accuracy: 0.9355 - val_loss: 1.0568 - val_accuracy: 0.7022 - val_top@3_accuracy: 0.9146
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇███████████████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇▇█████████████████████████
epoch/val_accuracy,▁▃▄▆▅▅▄▆▇▆▆▆▇▆▅▇▇▇▇█▆▇▇▇▇█▇████▇▇█▇▇▇▆█
epoch/val_loss,█▇▆▃▄▄▆▃▂▃▃▄▂▃▇▂▂▂▂▁▃▂▂▂▂▂▂▁▁▁▁▂▂▁▃▂▂▃▂
epoch/val_top@3_accuracy,▁▄▆▆▆▇▆▇▇▇▇▆▇▇▄█▇▇▇█▇█▇█▇█▇███████▇▇▇▇█
test_acc,▁


wandb: Agent Starting Run: d7n93sxp with config:
wandb: 	augmentation: heavy
wandb: 	batch_size: 64
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	kernel_size: 5x5
wandb: 	learning_rate: 0.001
wandb: 	net_filter_size: 32
wandb: 	net_n: 3
wandb: 	normalization: False
wandb: 	reg_alpha: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
704/704 [==============================] - 58s 79ms/step - loss: 1.7883 - accuracy: 0.3989 - top@3_accuracy: 0.7386 - val_loss: 2.3448 - val_accuracy: 0.3154 - val_top@3_accuracy: 0.6612
Epoch 2/100
704/704 [==============================] - 56s 80ms/step - loss: 1.4625 - accuracy: 0.5222 - top@3_accuracy: 0.8320 - val_loss: 1.8753 - val_accuracy: 0.4516 - val_top@3_accuracy: 0.8014
Epoch 3/100
704/704 [==============================] - 56s 80ms/step - loss: 1.2919 - accuracy: 0.5923 - top@3_accuracy: 0.8695 - val_loss: 1.6621 - val_accuracy: 0.5316 - val_top@3_accuracy: 0.7992
Epoch 4/100
704/704 [==============================] - 56s 80ms/step - loss: 1.1879 - accuracy: 0.6376 - top@3_accuracy: 0.8929 - val_loss: 1.6771 - val_accuracy: 0.5336 - val_top@3_accuracy: 0.8032
Epoch 5/100
704/704 [==============================] - 57s 80ms/step - loss: 1.1306 - accuracy: 0.6647 - top@3_accuracy: 0.9063 - val_loss: 1.1752 - val_accuracy: 0.6596 - val_top@3_accuracy: 0.8990
Epoch

best_val_acc,▁
best_val_loss,▁
epoch/accuracy,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/top@3_accuracy,▁▄▅▆▆▇▇▇▇▇▇▇▇▇▇█████████████████████████
epoch/val_accuracy,▁▃▄▄▆▆▆▅▆▂▇▇▇▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇██▇▇█▇▇▇██
epoch/val_loss,▆▅▄▄▃▂▂▃▂█▂▂▂▂▃▂▁▂▂▁▂▂▁▁▁▁▂▂▁▁▁▁▂▂▁▁▂▂▁▁
epoch/val_top@3_accuracy,▁▄▄▄▅▇▇▆▇▅▇▇█▇▇▇██▇▇▇▇████▇████▇▇▇██▇▇██
test_acc,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
